# Challenge 03: Minimum Makespan with Quantum Annealing  
  
**Solution done by [Billy.Ljm](https://github.com/BillyLjm/QOSF-Monthly-Challenges)**  
  
### Introduction  
  
The core of this challenge is about translating a real-world problem into a mathematical representation that fits a quantum computer. In this challenge we encourage you to use two representations which even though equivalent, are used in different contexts.  
  
The first is Quadratic Unconstrained Binary Optimization (QUBO) which is used primarily for the quantum annealers (D-Wave) and the other is Ising Hamiltonian, which is often used with an algorithm called Quantum Approximate Optimization Algorithm (QAOA).  
  
This challenge might require spending some time going through the references (especially 1-3), as we are not aware of any "quick introductions" worth mentioning. If you have a hard time understanding something, feel free to reach out for help on [QOSF Slack](https://join.slack.com/t/qosf/shared_invite/zt-bw59w8b9-WJ~k0~FAMHukTZov4AnLfA).  
  
Unless you have a prior background with these kinds of problems, we encourage you to start from QUBO and then perhaps trying Ising Hamiltonians as well, as there are more materials about it.  
  
It is also worth mentioning that D-Wave allows you to use their machines for free for a limited amount of time, so you can try running your QUBO on a real machine after signing up [here](https://cloud.dwavesys.com/leap/signup/).  
  
### Problem  
  
Niranjan owns a catering service and receives orders one day in advance. Each of the chefs he employs is paid by the hour. In order to optimize the cost of running his business, every night, he looks at all of the dishes he needs to deliver for the next day and asks his employees to work the specific hours he requires them for.  
  
Given an order queue ($Q$) of length $N$ with each order $i$ taking $L_i$ time to prepare, if there are $m$ chefs, design a QUBO/Ising Hamiltonian to help Niranjan find the shortest time ($T$) it would take for the $m$ chefs to prepare all $N$ dishes. Assume each chef can only work on one dish at a time and each dish is worked on by only one chef. We can also safely assume that at any given point, all chefs can be actively working on a dish.  
  
Since Niranjan wants to ensure that he can run his algorithms on the near term devices, keep track and try to minimize the number of:  
1. Gates & Qubits (for Ising Hamiltonian)  
2. Variables (for QUBO. For embedding on a real machine also keep track of the lengths of chains)  
  
**Optional:**  
For the advanced reader, you can also attempt to build the Cost Hamiltonian or Mixer Hamiltonian as described in [7,8].  
  
### Examples  
  
If $m = 1$, Order Queue = $\{2, 3, 7, 2, 1\}$  
then $T = 15$, with the allocation $m_1 = \{2, 3, 7, 2, 1\}$  
  
If $m = 2$, Order Queue = $\{2, 3, 7, 2, 1\}$  
then $T = 8$, with the allocation $m_1 = \{2, 3, 2\}$, $m_2 = \{7, 1\}$  
  
If $m = 3$, Order Queue = $\{2, 3, 2, 2, 1\}$  
then $T = 4$, with the allocation $m_1 = \{3\}$, $m_2 = \{2, 2\}$, $m_3 = \{2, 1\}$  
  
### Resources  
  
\[1\] [D-Wave Problem Solving Handbook](https://docs.dwavesys.com/docs/latest/doc_handbook.html)  
\[2\] [Implementation of the Travelling Salesman Problem](https://github.com/mstechly/quantum_tsp_tutorials)  
\[3\] [Ising formulations of many NP problems](https://arxiv.org/pdf/1302.5843.pdf)  
\[4\] [D-Wave Examples](https://docs.ocean.dwavesys.com/en/stable/getting_started.html#examples)  
\[5\] [Quantum Approximate Optimization Algorithm explained](https://www.mustythoughts.com/quantum-approximate-optimization-algorithm-explained)  
\[6\] [Job Shop Scheduling Solver based on Quantum Annealing](https://arxiv.org/pdf/1506.08479.pdf)  
  
### References  
  
\[7\] [Quantum Algorithms for Scientific Computing and Approximate Optimization](https://arxiv.org/pdf/1805.03265.pdf)  
\[8\] [From the Quantum Approximate Optimization Algorithm to a Quantum Alternating Operator Ansatz](https://arxiv.org/pdf/1709.03489.pdf)  

## My Approach  
The job allocation is represented with qubits, with $q_{j,m} = 1$ denoting that the job $j$ has been allocated to machine $m$.  
And the weight/duration of each job $w_j$ is encoded in the QUBO/Ising coefficients.  
Additionally, we use QUBO formalism since it is more convenient, where $q_{m,j} \in \{0, 1\}$ <sub>*(translatable to Ising via $s = 2x - 1$)*</sub>  
  
We first have to ensure each job is assigned to only 1 machine, without bias.  
This is can be encouraged via the minimisation of the following polynomial.  
  
$$E_0 = \sum_j \left[ -\sum_m q_{j,m} + \sum_{a \in \{m\}} \sum_{b \in \{m\} \setminus a} q_{j,a} q_{j,b} \right]$$  
  
Next, we want to distribute the jobs as evenly across the machines, in terms of time to complete.  
My approach to do this is to minimise the variance $\sigma^2$ in the length of assigned job queues, as encapsulated in  
  
$$\begin{aligned}  
    E_1  
    &= \sigma^2 = \sum_m \left( \sum_j w_j q_{j,m} - \bar{w} \right)^2 \text{ ,where } \bar{w} = \frac{\sum_j w_j}{\text{len}(m)}\\  
    &= \sum_m \left(\begin{aligned}  
        &\bar{w}^2 + \sum_j w_j^2 q_{j,m} - 2 \sum_j \bar{w} w_j q_{j,m}\\  
        &+ \sum_{x \in \{j\}} \sum_{y \in \{j\} \setminus x} w_x w_y q_{x,m} q_{y,m}\\  
    \end{aligned}\right)  
\end{aligned}$$  
  
The square $(\ldots)^2$ is critical to finding the correct output,  
Since it penalises a small number of large length differences more than a large number of small differences  
Specifically, it will never be preferable to transfer a job of length $k$ from a shorter queue $\bar{w} + m$ to a larger queue $\bar{w} + n$ <sub>*(it doesn't apply w/o square)*</sub>  
  
$$n^2 + m^2 \lt (n+k)^2 + (m-k)^2 \ ,\forall \ n \geq m \cup k \gt 0$$  
  
Lastly, we must ensure that assigning 1 job to 1 machine take precedence over any possible variance in the optimal output.  
This can be done by combining the contributions above into the final QUBO polynomial below. <sub>*(ignoring the constant $\bar{w}^2$)*</sub>  
  
$$\begin{aligned}  
    E_{\mathrm{tot}}  
    &= \sigma^2_{max} E_0 + E_1 \text{ ,where } \sigma^2_{max} = \left( \sum_j w_j - \bar{w} \right)^2\\  
    &= \left[\begin{aligned}  
        &\sum_j \sum_{a \in \{m\}} \sum_{b \in \{m\} \setminus a} \sigma^2_{max} q_{j,a} q_{j,b}\\  
        +& \sum_m \sum_{x \in \{j\}} \sum_{y \in \{j\} \setminus a} w_x w_y q_{x,m} q_{y,m}\\  
        +& \sum_j \sum_m (-\sigma^2_{max} + {w_j}^2 - 2 \bar{w} w_j) q_{j,m}\\  
    \end{aligned}\right]  
\end{aligned}$$  

## D-Wave Implementation
I decided to implement the solution using D-Wave SDK, since their quantum annealing approach is most suited for QUBO.  
I implemented the QUBO directly using the most basic `BinaryQuadraticModel`, since the problem has already been formulated above.  

In [1]:
import numpy as np
from dimod import BinaryQuadraticModel, ExactSolver
from dwave.system import LeapHybridSampler

np.set_printoptions(precision=3)

In [2]:
def minmakespan(machines, jobs, quantum=False):
    """Finds minimum makespan of a jobs on a number of machines

    Paramters:
        machines (int): number of machines to do jobs
        jobs (array): array of the durations of each job
        quantum (bool): True to solve on D-Wave annealers,
                        False to solve via local classical simulation

    Returns:
        array of job schedules, with each schedule being a 2D array with
        schedule[job][machine] = 1 meaning the job is assigned to the machine
    """
    # init variables
    njobs = len(jobs)
    wbar = sum(jobs) / machines
    varmax = (sum(jobs) - wbar)**2
    varmax = max(varmax, 1) # to avoid varmax = 0

    # qubit indexing function
    def q(j,m):
        return j * machines + m

    # specficy QUBO coefficients
    linear = {}
    quadratic  = {}
    for j in range(njobs):
        for a in range(machines):
            for b in range(a+1, machines):
                # note we iterate b > a, instead of b != a
                # thus we have an extra factor of 2 (vs eqn above)
                quadratic[(q(j,a), q(j,b))] = 2 * varmax
    for m in range(machines):
        for x in range(njobs):
            for y in range(x+1, njobs):
                # note we iterate b > a, instead of b != a
                # thus we have an extra factor of 2 (vs eqn above)
                quadratic[(q(x,m), q(y,m))] = 2 * jobs[x] * jobs[y]
    for j in range(njobs):
        for m in range(machines):
            linear[q(j,m)] = -varmax + jobs[j]**2 - 2 * wbar * jobs[j]

    # solve DQM
    bqm = BinaryQuadraticModel(linear, quadratic, vartype='BINARY')
    sampleSet = LeapHybridSampler().sample(bqm) if quantum else ExactSolver().sample(bqm)

    # translate results into job assignemtns
    results = sampleSet.lowest()
    results = results.record.sample
    out = np.resize(results, (len(results), njobs, machines))
    return(out)

In [3]:
def makespan(schedule, jobs):
    """Calculate makespan of a schedule in the format of
    schedule[job][machine] = 1 meaning the job is assigned to the machine

    Parameters:
        schedule (array): list of job assignments with job-machine
                          assignments denoted by schedule[job][machine] = 1
        jobs (array): array of the durations of each job

    Returns:
        maximum length of job queues assigned to the machines
    """
    # init variables
    machines = len(schedule[0])
    njobs = len(jobs)

    # calculate queue lengths
    qlen = [0] * machines
    for j in range(njobs):
        for m in range(machines):
            qlen[m] += schedule[j][m] * jobs[j]

    return max(qlen)

## Testing
To show that the QUBO formulation above works, we can apply it for a random number of machines, jobs & job lengths  
Note that runninng on D-Wave hardware, via `minmakespan(..., quantum=True)`, requires a [D-Wave leap](https://www.dwavesys.com/take-leap) account  
Alternatively, the algorithm can be run locally via classical simulation using `minmakespan(..., quantum=False)`  

In [4]:
import random

In [5]:
n = 10 # range for random variables

# generate random input
machines = random.randint(1,n)
jobs = []
for _ in range(2 * random.randint(1,n)):
    jobs.append(random.random() * n)

# minimise makespan
schedules = minmakespan(machines, jobs, quantum=True)
mspan = makespan(schedules[0], jobs)

# print data
print("machines: %d" % machines)
print("jobs: ", np.array(jobs))
print("makespan: %.3f" % mspan)
print("schedule:\n", schedules[0])

machines: 3
jobs:  [6.059 8.709 8.547 4.692 4.214 2.417 3.231 8.682 9.028 3.201 2.973 1.225
 0.013 7.284]
makespan: 23.582
schedule:
 [[0 1 0]
 [0 1 0]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [0 1 0]
 [0 0 1]
 [0 1 0]]
